# If there are contents in DynamoDB, delete items

In [1]:
# import boto3

# region = 'us-west-2'

# table_name = "speech2text"
# dynamodb_client = boto3.resource('dynamodb',region_name='us-west-2')
# table = dynamodb_client.Table(table_name)

# # Scan the table to get all the items
# scan = table.scan()
# with table.batch_writer() as batch:
#     while 'Items' in scan and len(scan['Items']) > 0:
#         for item in scan['Items']:
#             # print(item)
#             # Delete each item
#             batch.delete_item(Key={'ContactID': item['ContactID'], 'SaidTimeStamp': item['SaidTimeStamp']})
#         # Scan for more items (DynamoDB can limit the number of items returned by scan, use 'ExclusiveStartKey' to continue scanning)
#         if 'LastEvaluatedKey' in scan:
#             scan = table.scan(ExclusiveStartKey=scan['LastEvaluatedKey'])
#         else:
#             break

# print(f"All items in the table '{table_name}' have been deleted.")

# Do inference using faster whisper endpoint 

In [3]:
import boto3

def list_s3_contents(bucket_name, prefix):
    # Initialize a session using Amazon S3
    s3 = boto3.client('s3')

    # Initialize the variables
    continuation_token = None
    contents = []

    while True:
        if continuation_token:
            response = s3.list_objects_v2(
                Bucket=bucket_name, 
                Prefix=prefix,
                ContinuationToken=continuation_token
            )
        else:
            response = s3.list_objects_v2(
                Bucket=bucket_name, 
                Prefix=prefix
            )

        if 'Contents' in response:
            contents.extend(response['Contents'])

        if response.get('IsTruncated'):  # if the response is truncated, there are more keys to retrieve
            continuation_token = response.get('NextContinuationToken')
        else:
            break

    return contents

# Define the bucket name and prefix
# s3_uri = 's3://sagemaker-us-west-2-452145973879/datasets/midea_data/midea_dialogue/short_30s/'
s3_uri = 's3://sagemaker-us-west-2-452145973879/datasets/midea_data/midea_dialogue/short/'

bucket_name = s3_uri.split('/')[2]
prefix = '/'.join(s3_uri.split('/')[3:])

# List the contents
contents = list_s3_contents(bucket_name, prefix)
# for obj in contents:
#     print(obj['Key'])

print(len(contents))
contents[2]

1310


{'Key': 'datasets/midea_data/midea_dialogue/short/0e46d005-4c13-4300-add0-683761ad3ebd_100_agent.wav',
 'LastModified': datetime.datetime(2024, 6, 18, 6, 36, 30, tzinfo=tzlocal()),
 'ETag': '"51473fb6a3143464373a336d57709039"',
 'Size': 160044,
 'StorageClass': 'STANDARD'}

## For demo, only keep 3 dialogues

In [ ]:
# dialogue_ids = [
#     '3ee4b9f4-7674-4978-9066-d89b46c9adb4',
#     '8fcf8eb4-d4da-4146-a42c-d3ba711b4ed0',
#     'aec95ea8-e448-45a6-be21-63a6b6326b9f',
# #    '942df28b-6803-4795-ab8b-7cba16a92ea8'
# ]

# contents_demo = []

# for content in contents:
#     for dialogue_id in dialogue_ids:
#         if dialogue_id in content['Key']:
#             contents_demo.append(content)
#             break

# print(len(contents_demo))
# contents_demo[0]

contents_demo = contents

In [ ]:
# import json
# import urllib.parse
# import boto3
# import os
# import logging
# from tqdm import tqdm

# import sagemaker
# import boto3
# # Get the SageMaker session and default S3 bucket
# # sagemaker_session = sagemaker.Session()
# sagemaker_session = sagemaker.session.Session()
# bucket = sagemaker_session.default_bucket()
# region = sagemaker_session._region_name
# role = sagemaker.get_execution_role()
# print(f"bucket: {bucket}, role:{role}, region: {region}")

# #dynamodb_client = boto3.resource('dynamodb',region_name='us-west-2')
# #s3_client = boto3.client('s3',region_name='us-west-2')
# sagemaker_runtime= boto3.client('runtime.sagemaker',region_name=region)
# logger = logging.getLogger()
# logger.setLevel("INFO")
# #conversation_table = dynamodb_client.Table("speech2text")
# s3_bucket = bucket
# endpoint_name = 'faster-whisper-hf-real-time-endpoint-1719447623'

# num_test = min(10000, len(contents_demo))
# for content in tqdm(contents_demo[:num_test], total=num_test):
#     key = urllib.parse.unquote_plus(content['Key'], encoding='utf-8')
#     try:
#         logger.info("s3_bucket:"+s3_bucket)
#         logger.info("key:"+key)
#         data = json.dumps({"s3_bucket":s3_bucket,"key":key})
#         response = sagemaker_runtime.invoke_endpoint(EndpointName=endpoint_name,ContentType='application/json',Body=data)
#             #logging("type:"+type(response))
#             #logging("response:"+str(response))
#             #results = json.loads(response['Body'].read().decode())
#             #response = s3_client.get_object(Bucket=s3_bucket, Key=key)
#         '''
#             with conversation_table.batch_writer() as batch:
#                 for item in results:
#                     print(item)
#                     response = batch.put_item(Item={
#                     "ContactID": item["contact_id"],
#                     "SaidTimeStamp": item["time_stamp"],
#                     "Speaker": item["speaker"],
#                     "Words": item["words"]
#                     })
#             #return response['ContentType']
#         '''
#         #return "success"
#         # print(response)
#     except:
#         logger.info("ignore known defects")

# Read data from DynamoDB

In [4]:
import boto3

# Initialize a session using Amazon DynamoDB
dynamodb = boto3.resource('dynamodb')

# Specify the DynamoDB table
table_name = 'speech2text'
table = dynamodb.Table(table_name)

# Scan the table
def scan_table(table):
    response = table.scan()
    data = response['Items']

    while 'LastEvaluatedKey' in response:
        response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
        data.extend(response['Items'])
    
    return data

# Get items from the table
items = scan_table(table)

# # Print the items
# for item in items:
#     print(item)
print(len(items), items[0])

2071 {'ContactID': '47627d48-9452-4639-8d27-154b61a3a185', 'SaidTimeStamp': Decimal('0'), 'Words': '執行執勢經營', 'Speaker': 'customer'}


In [5]:
import pandas as pd
from decimal import Decimal

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(items)
# Sort the DataFrame by 'ContactID' and 'SaidTimeStamp'
df = df.sort_values(by=['ContactID', 'SaidTimeStamp'])

# Group by 'ContactID' and concatenate 'Words' with 'Speaker' prefix
grouped = df.groupby('ContactID').apply(
    lambda x: ''.join(f"{row['Speaker']}: {row['Words']}\n" for _, row in x.iterrows())
).reset_index(name='content')

# Rename the columns to match the image
ckpt_name = 'v7_ckpt60'
grouped.columns = ['contact_id', f'content_{ckpt_name}']

dialogue_col = f'content_{ckpt_name}'

df = grouped
df.head()

/tmp/ipykernel_9491/327332243.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = df.groupby('ContactID').apply(


,contact_id,content_v7_ckpt60
0,0e46d005-4c13-4300-add0-683761ad3ebd,agent: 唔講東區服務部喎姓楊有咩幫到你呀\ncustomer: 你好呀楊小姐我姓葉㗎\...
1,11383068-87cc-4d16-a7b2-bc5a518c7472,agent: 你，係你好通知\ncustomer: 係你好\ncustomer: 係我屋企嗰...
2,3ee4b9f4-7674-4978-9066-d89b46c9adb4,agent: 早晨都稍後復過性樂嘅咁吖吖\ncustomer: 喂係係呀我想問問呢我有一個嗰...
3,47627d48-9452-4639-8d27-154b61a3a185,customer: 執行執勢經營\ncustomer: 執行執勢經營\ncustomer: ...
4,53b4c19b-4042-46ba-a4e6-a1893f4704d4,agent: 有咩幫到你\ncustomer: 唔該我屋企有個蒸氣焗爐呀\nagent: 係...


# Extract infomation from dialogue

# Qwen2 series

## Qwen2-72B-Instruct-AWQ

In [ ]:
#%pip install openai

In [6]:
## deploy using vllm
# Set OpenAI's API key and API base to use vLLM's API server.
from openai import OpenAI

openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"
model_id = "Qwen2-72B-Instruct-AWQ"

stop_token_ids = ''
temp = 0.8

# Create an OpenAI client to interact with the API server
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

def predict(message, history):
    # Convert chat history to OpenAI format
    history_openai_format = [{
        "role": "system",
        "content": "You are a great ai assistant."
    }]
    for human, assistant in history:
        history_openai_format.append({"role": "user", "content": human})
        history_openai_format.append({
            "role": "assistant",
            "content": assistant
        })
    history_openai_format.append({"role": "user", "content": message})

    # Create a chat completion request and send it to the API server
    stream = client.chat.completions.create(
        model=model_id,  # Model name to use
        messages=history_openai_format,  # Chat history
        temperature=temp,  # Temperature for text generation
        # stream=True,  # Stream response
        extra_body={
            'repetition_penalty':
            1,
            'stop_token_ids': [
                int(id.strip()) for id in stop_token_ids.split(',')
                if id.strip()
            ] if stop_token_ids else []
        })

    return stream

In [7]:
# SageMaker Endpoint
import json

def invoke_endpoint(predictor, tokenizer, prompt, system_prompt):

    # prompt = """'下面是一段agent与customer的对话\nagent: 咁嘅\ncustomer: 你好呀,我正在用你們的抽濕機\ncustomer: 用咗半年到啦咁而家呢佢壞咗呀佢呢\nagent: 嗨\ncustomer: 濕濕咗嘅時候呢嗰啲水呢直接流落去地下\ncustomer: 接不到那些水\nagent: 請問點稱呼呀?\nagent: 余先生,請問你可否給我抽濕口罩?\ncustomer: 我姓余的\nagent: 手機的型號\ncustomer: RADY200H\nagent: 多謝大家\ncustomer: Thank you for watching\nagent: OK\ncustomer: 拜拜\nagent: 又係咪check我嗰個水準\nagent: 放入去嘅時候,位置係正常\nagent: 唔好擺尾之類\ncustomer: 冇嘅冇排名\nagent: Check過冇問題嘅,如果係可以搵錢\nagent: 咁請問你嗰個抽濕機係買咗一年到?嘛係咪?\nagent: 上年九月抽濕機\ncustomer: 上年九月買的\nagent: 请问您的购买单和保用证是否存在?\ncustomer: 喺度嘅\nagent: 不存在的,如果我可以找师傅上来帮你看,麻烦你出示给他\nagent: 請問您的地址在哪裡?\ncustomer: 柴灣道111號\ncustomer: 高威港\nagent: 高威國幾座幾樓幾室\ncustomer: 第四座,四座八樓私宅\nagent: SipoCat\ncustomer: 係冇差\nagent: 麻煩你等一等我睇下如果係柴灣嘅可以幾時安排到過嚟先\ncustomer: 唔\nagent: 嚟緊星期二 23 號 12 點,\nagent: 屋企會唔會有忍\ncustomer: 哎呀\nagent: 咁我呢度幫你安排返啦係余生嘅到時上嚟之前師傅可以打返90997\nagent: 910 呢個電話高威國四座扮cat嘅啱啱\ncustomer: 我留兩個電話給你\nagent: 好呀你再講呀\ncustomer: 我太太梁小姐\nagent: 90256922,呢個係搵邊位??\nagent: 即係阿梁小姐\nagent: 打誰的電話先?\nagent: 由我講吧!\ncustomer: 你打這個先啦,我再留多個屋企電話比你啦\ncustomer: 以5687021\nagent: 屋企電話25687021,如果係到時打電話嘅時候,可以搵返梁小姐係\nagent: 902-56922\ncustomer: 係冇錯\nagent: 好得,我呢度幫你安排返\nagent: 23 號 12 點至 5 點過嚟\ncustomer: 星期五至五點,星期二。\nagent: 係,冇錯,有咩其他可以幫到你?\ncustomer: OK,好呀,\nagent: 都係打電話嚟,多謝!\ncustomer: 係冇?啦,\nagent: 拜拜\ncustomer: OK\n\n\n请从上面的对话中抽取如下信息，并以json格式返回，如果对话中没有提到相应字段的内容，则填""：\n {"customerType": "个人", "customerName": "溫先生", "phoneNumber1": "", "phoneNumber2": "", "email": "", "address": "楊逸居第三座,九樓,A7", "productBrand": "Toshiba", "productCategoryName": "", "serialNumber": "ER-GD400HK", "srType": "维修", "srSubType": "维修", "symptomDescription": "燈膽燒了", "customerRequest": "維修法", "refNo": "", "selloutInvoiceNum": "", "salesDealerName": "", "installDealerName": ""}\n \n注意只返回抽取的json格式的结果，不返回其它额外信息。 \n'"""
    # system_prompt = "You are a helpful assistant."
    # system_prompt = "你是一位资深的客服中心坐席经理，你的任务是从给定的对话中抽取要求的关键信息并按要求的格式返回结果。"

    messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
    ]

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    response = predictor.predict(
        {
            "inputs": prompt, 
             "parameters": 
             {
                 "max_new_tokens": 4096,
                 # Add any other sampling parameters as needed
                 "temperature": 0.7,
                 "top_k": 5,
                 "top_p": 0.9,
                 # "stop_token_ids": [], 
                 # "stop": ["\nASSISTANT", "\nUSER:"],
                 "include_stop_str_in_output": False,
                 # "skip_special_tokens": True,
                 "ignore_eos": False,
                 "repetition_penalty": 1,
             }
        }
    )
    
    generated_text = json.loads(response.decode('utf-8'))['generated_text']
    
    return generated_text

In [8]:
# from transformers import AutoTokenizer
# import sagemaker
# from sagemaker import Model, image_uris, serializers, deserializers
# from tqdm import tqdm
# import time
# import json

# role = sagemaker.get_execution_role()  # execution role for the endpoint
# sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
# region = sess._region_name  # region name of the current SageMaker Studio environment
# account_id = sess.account_id()  # account_id of the current SageMaker Studio environment

# endpoint_name = 'lmi-model-2024-06-20-02-23-51-303'

# predictor = sagemaker.Predictor(
#     endpoint_name=endpoint_name, 
#     sagemaker_session=sess,
#     serializer=serializers.JSONSerializer()
# )

# model_dir = '/home/ec2-user/SageMaker/efs/Models/Qwen2-72B-Instruct-AWQ'
# tokenizer = AutoTokenizer.from_pretrained(model_dir)

# model_name = 'qwen2-72b'

# num_test = 100
# for i, row in tqdm(df[:num_test].iterrows(), total=df[:num_test].shape[0]):
#     content = row[dialogue_col]
    
# #     prompt = f"""下面是一段agent与customer的对话
# # {content}

# # 请从上面的对话中抽取如下信息，并以json格式返回，如果对话中没有提到相应字段的内容，则填""：
# #  {{"customerType": "个人", "customerName": "溫先生", "phoneNumber1": "", "phoneNumber2": "", "email": "", "address": "楊逸居第三座,九樓,A7", "productBrand": "Toshiba", "productCategoryName": "", "serialNumber": "ER-GD400HK", "srType": "维修", "srSubType": "维修", "symptomDescription": "燈膽燒了", "customerRequest": "維修法", "refNo": "", "selloutInvoiceNum": "", "salesDealerName": "", "installDealerName": ""}}
 
# # 注意只返回抽取的json格式的结果，不返回其它额外信息。 
# # """
#     prompt = f"""这是一个命名实体识别任务，需要你参考<样例>，对给定的<文本>(包含了一段agent与customer的对话)抽取对应的实体，注意只返回抽取的json结果，不要返回任何其它额外信息。

# <样例>
# 输入：{example}
# 输出：{{"customer_type": "Person","sr_type": "Repair", "sr_sub_type": "Repair", "last_name": "余先生", "country_code": "HK", "phone_number1": "90997910", "phone_number2": "<NULL>", "email": "<NULL>", "state_name": "港島", "city_name": "東區", "address": "港島 東區 柴灣 高威閣 4座 8樓 C室", "reference_num": "<NULL>", "product_brand": "TOSHIBA", "product_category": "<NULL>", "symptom_description": "機件故障", "customer_request": "<NULL>"}}

# <文本>
# 输入：{content}
# 输出：
# """
    
#     try:
#         start_time = time.time()

#         ## vllm direct deployment
#         response = predict(prompt, [])
#         generated_text = response.choices[0].message.content

#         # # sagemaker endpoint deployment
#         # generated_text = invoke_endpoint(predictor, tokenizer, prompt)

#         time_cost = time.time() - start_time

#         df.loc[i, f'extracted_info_{model_name}'] = generated_text
#         res = json.loads(generated_text)
#         if isinstance(res, dict):
#             for k, v in res.items():
#                 df.loc[i, f"{k}_{model_name}"] = v

#         df.loc[i, f'time_cost_{model_name}'] = time_cost
    
#     except Exception as e:
#         print(f"Exception: {e}")
#         print(f"contact_id: {row['contact_id']}")
#         print(f"result: {generated_text}")
# df.head()

## Qwen2-7B-Instruct

In [9]:
from transformers import AutoTokenizer
import sagemaker
from sagemaker import Model, image_uris, serializers, deserializers
from tqdm import tqdm
import time
import json
import re

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
region = sess._region_name  # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment

endpoint_name = 'Qwen2-7B-Instruct-2024-06-25-03-29-20-049'

predictor = sagemaker.Predictor(
    endpoint_name=endpoint_name, 
    # sagemaker_session=sess,
    serializer=serializers.JSONSerializer()
)

model_dir = '/home/ec2-user/SageMaker/efs/Models/Qwen2-7B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_dir)

model_name = 'qwen2-7b'

num_test = min(100, df.shape[0])
for i, row in tqdm(df[:num_test].iterrows(), total=num_test):
    content = row[dialogue_col]
    
#     prompt = f"""下面是一段agent与customer的对话
# {content}

# 请从上面的对话中抽取如下信息，并以json格式返回，如果对话中没有提到相应字段的内容，则填""：
#  {{"customerType": "个人", "customerName": "溫先生", "phoneNumber1": "", "phoneNumber2": "", "email": "", "address": "楊逸居第三座,九樓,A7", "productBrand": "Toshiba", "productCategoryName": "", "serialNumber": "ER-GD400HK", "srType": "维修", "srSubType": "维修", "symptomDescription": "燈膽燒了", "customerRequest": "維修法", "refNo": "", "selloutInvoiceNum": "", "salesDealerName": "", "installDealerName": ""}}
 
# 注意只返回抽取的json格式的结果，不返回其它额外信息。 
# """

#     prompt = f"""这是一个命名实体识别任务，需要你参考<样例>，对给定的<文本>(包含了一段agent与customer的对话)抽取对应的实体 。

# <样例>
# 输入：{example}
# 输出：{{"customer_type": "Person","sr_type": "Repair", "sr_sub_type": "Repair", "last_name": "余先生", "country_code": "HK", "phone_number1": "90997910", "phone_number2": "<NULL>", "email": "<NULL>", "state_name": "港島", "city_name": "東區", "address": "港島 東區 柴灣 高威閣 4座 8樓 C室", "reference_num": "<NULL>", "product_brand": "TOSHIBA", "product_category": "<NULL>", "symptom_description": "機件故障", "customer_request": "<NULL>"}}

# <文本>
# 输入：{content}
# 输出：
# """
    # system_prompt = "你是一位资深的客服中心坐席经理，你的任务是从给定的对话中抽取要求的关键信息并按要求的格式返回结果。"
    
# 输出：{{"sr_type": "Repair", "last_name": "余", "phone_number1": "90997910", "state_name": "港島", "city_name": "東區", "address": "港島 東區 柴灣 高威閣 4座 8樓 C室", "product_brand": "TOSHIBA", "product_category": "焗爐", "symptom_description": "機件故障", "summary": "..."}}

#     prompt = f"""\
# 请从下面customer和agent的对话中，提取如下信息

# 1. sr_type: customer的请求类型，根据对话内容从 [Repair, Inquiry, Installation] 中选择一个；
# 2. last_name: customer 的姓，需要加上"先生"、"女士"的称呼，例如 "余先生"，"叶女士"；
# 3. phone_number1: customer 的电话号码，请用阿拉伯数字返回；
# 4. state_name: customer 地址的 state, 例如 "港島"；
# 5. city_name: customer 地址的 city, 例如 "東區"；
# 6. address: customer 的详细地址；
# 7. product_category: 产品的种类，例如 "焗爐"；
# 8. product_brand: 产品的品牌，例如 "TOSHIBA"；
# 9. serial_number: 产品型號；
# 10. symptom_description: 产品的问题描述，例如 "燈膽燒壞"；
# 11. summary: 对话的简单总结，限50字。

# 注意：
# 1. 如果对话中没有提及，以""填充，不要编造；
# 2. 提取的信息以json格式返回，并且只返回json格式的结果，不返回任何其它信息。

# 输入：{content}
# 输出：
# """

    system_prompt = "你係一位资深嘅客服中心坐席经理，你嘅任务係从畀定嘅对话中抽取要求嘅关键信息并按要求嘅格式返回结果。"

    prompt = f"""\
请从下面 customer 和 agent 嘅对话中，提取如下信息：

1. sr_type: customer 嘅请求类型，根據对话内容从 [Repair, Inquiry, Installation] 中揀一个；
2. last_name: customer 嘅姓，需要加上 "先生"、"女士" 嘅称呼，例如 "余先生"，"叶女士"；
3. phone_number1: customer 嘅电话号码，请用阿拉伯数字返回；
4. state_name: customer 地址嘅 state；
5. city_name: customer 地址嘅 city；
6. address: customer 嘅详细地址；
7. product_category: 产品嘅种类"；
8. product_brand: 产品嘅品牌；
9. serial_number: 产品型号；
10. symptom_description: 产品嘅问题描述；
11. summary: 对话嘅简单总结，限 50 字。

注意：
1. 如果对话中冇提及，以 "" 填充，唔好編造；
2. 提取嘅信息以 JSON 格式返回，并且只返回 JSON 格式嘅结果，唔返回任何其他信息。

输入：{content}
输出：
"""
    
    try:
        start_time = time.time()

        ## vllm direct deployment
        # response = predict(prompt, [])
        # generated_text = response.choices[0].message.content

        # sagemaker endpoint deployment
        generated_text = invoke_endpoint(predictor, tokenizer, prompt, system_prompt)
        # print(type(generated_text), generated_text)

        time_cost = time.time() - start_time

        df.loc[i, f'extracted_info_{model_name}'] = generated_text
        json_text = re.search(r'{.*}', generated_text, re.DOTALL).group(0)
        print(json_text)
        
        res = json.loads(json_text)
        if isinstance(res, dict):
            for k, v in res.items():
                df.loc[i, f"{k}_{model_name}"] = v

        df.loc[i, f'time_cost_{model_name}'] = time_cost
    except Exception as e:
        print(f"Exception: {e}")
        print(f"contact_id: {row['contact_id']}")
        print(f"result: {generated_text}")

/home/ec2-user/SageMaker/efs/conda_envs/qwen_py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
  6%|▌         | 1/18 [00:05<01:39,  5.83s/it]

{
  "sr_type": "Inquiry",
  "last_name": "葉女士",
  "phone_number1": "",
  "state_name": "",
  "city_name": "",
  "address": "",
  "product_category": "飯煲",
  "product_brand": "",
  "serial_number": "RC-18NMF IH-WT",
  "symptom_description": "煮飯唔夠唔甜，出現C14錯誤，產品問題",
  "summary": "葉女士購買一個月的蘇寧飯煲出現煮飯不夠甜及C14錯誤，經多次嘗試後問題未解決，疑責產品問題。"
}


 11%|█         | 2/18 [00:11<01:28,  5.55s/it]

...

{
  "sr_type": "Inquiry",
  "last_name": "鄭先生",
  "phone_number1": "",
  "state_name": "",
  "city_name": "",
  "address": "",
  "product_category": "洗碗碟機",
  "product_brand": "東芝",
  "serial_number": "",
  "symptom_description": "產品在使用後出現未完全風干問題，特別是「清爽」模式下，產品內部仍有水滴，影響衛生和使用體驗。",
  "summary": "顧客反映東芝洗碗碟機在「清爽」模式下未完全風干問題，產品內部仍有水滴，影響衛生和使用體驗。"
}


 94%|█████████▍| 17/18 [01:34<00:05,  5.42s/it]

{
  "sr_type": "Repair",
  "last_name": "未知先生",
  "phone_number1": "66504092",
  "state_name": "未知",
  "city_name": "未知",
  "address": "未知",
  "product_category": "洗衣機",
  "product_brand": "未知",
  "serial_number": "未知",
  "symptom_description": "發出怪聲，在入水時機器無法運行",
  "summary": "顧客報告洗衣機在入水時無法運行且發出怪聲，需要修理。"
}


100%|██████████| 18/18 [01:41<00:00,  5.61s/it]

{
  "sr_type": "Installation",
  "last_name": "秦女士",
  "phone_number1": "98331297",
  "state_name": "",
  "city_name": "粉嶺",
  "address": "華明邨信名樓一九一二",
  "product_category": "洗衣機",
  "product_brand": "AWM801",
  "serial_number": "APH",
  "symptom_description": "",
  "summary": "秦女士詢問關於其在三月五日於伊羅購買的AWM801型號洗衣機的安裝事宜，客服確認已收到貨物，並告知將於次日安排安裝，且因購買時間在三月內，享受免费安装服务。"
}


In [10]:
df.head()

,contact_id,content_v7_ckpt60,extracted_info_qwen2-7b,sr_type_qwen2-7b,last_name_qwen2-7b,phone_number1_qwen2-7b,state_name_qwen2-7b,city_name_qwen2-7b,address_qwen2-7b,product_category_qwen2-7b,product_brand_qwen2-7b,serial_number_qwen2-7b,symptom_description_qwen2-7b,summary_qwen2-7b,time_cost_qwen2-7b
0,0e46d005-4c13-4300-add0-683761ad3ebd,agent: 唔講東區服務部喎姓楊有咩幫到你呀\ncustomer: 你好呀楊小姐我姓葉㗎\...,"```json\n{\n ""sr_type"": ""Inquiry"",\n ""last_n...",Inquiry,葉女士,,,,,飯煲,,RC-18NMF IH-WT,煮飯唔夠唔甜，出現C14錯誤，產品問題,葉女士購買一個月的蘇寧飯煲出現煮飯不夠甜及C14錯誤，經多次嘗試後問題未解決，疑責產品問題。,5.828054
1,11383068-87cc-4d16-a7b2-bc5a518c7472,agent: 你，係你好通知\ncustomer: 係你好\ncustomer: 係我屋企嗰...,"```json\n{\n ""sr_type"": ""Repair"",\n ""last_na...",Repair,宋先生,,,,,Stove,Sogo,MS5TR630SC,无法启动，灯不亮，蒸煮功能无法使用，有水漏到桌面,客户报告其Sogo品牌蒸炉无法启动，灯不亮，且有水漏到桌面，询问维修事宜。,5.351527
2,3ee4b9f4-7674-4978-9066-d89b46c9adb4,agent: 早晨都稍後復過性樂嘅咁吖吖\ncustomer: 喂係係呀我想問問呢我有一個嗰...,"```json\n{\n ""sr_type"": ""Repair"",\n ""last_na...",Repair,溫先生,,,,九龍尖沙咀柯士尖道楊逸居第三座九樓LIP禁九字A室,微波爐,,ER-GD-400HK-XR,燈膽燒壞,顧客溫先生有一個微波爐的燈膽燒壞，需要維修。顧客提供地址為九龍尖沙咀柯士尖道楊逸居第三座九樓...,7.460177
3,47627d48-9452-4639-8d27-154b61a3a185,customer: 執行執勢經營\ncustomer: 執行執勢經營\ncustomer: ...,"```json\n{\n ""sr_type"": ""Repair"",\n ""last_na...",Repair,王先生,,,,,蒸氣焗爐,,,產品無水，但實際上有油，且在使用過程中出現漏水問題,顧客報告蒸氣焗爐在使用時出現無水問題，實際上產品內有油，且在之前的維修中存在漏水問題。,4.756153
4,53b4c19b-4042-46ba-a4e6-a1893f4704d4,agent: 有咩幫到你\ncustomer: 唔該我屋企有個蒸氣焗爐呀\nagent: 係...,"```json\n{\n ""sr_type"": ""Repair"",\n ""last_na...",Repair,李女士,,星谷路,,雨晴軒 审座廿九詩,蒸氣焗爐,,MS2PQ205C,不出蒸氣,客户李女士的蒸氣焗爐不出蒸氣，预约师傅上门维修，费用为330元，如果现场可以维修则现场报价，...,5.692189


In [11]:
df.to_excel("outputs/transcripts_demo.xlsx", index=False)